# personal_vault.notes
> Functions for managing notes in an Obsidian.md math vault

In [ ]:
#| default_exp personal_vault.notes

In [ ]:
#| export
from typing import Union

from trouver.obsidian.file import(
    MarkdownFile
)
from trouver.obsidian.links import (
    links_from_text
)
from trouver.obsidian.vault import (
    VaultNote
)


In [ ]:
from fastcore.test import *
from trouver.helper.tests import _test_directory

## Get notes linked in notes

In [ ]:
#| export
def notes_linked_in_note(
        list_note: VaultNote, # The `VaultNote` in which to find the links.
        as_dict: bool = True # If `True`, returns a dict. Returns a list otherwise.
        ) -> Union[dict[str, VaultNote], list[VaultNote]]: # If dict, the keys are the names of the vault notes and the values are the `VaultNote` objects. If list, then the entries are the `VaultNote` objects.
    """Returns a list or dict of VaultNotes of notes linked by 
    a specified note.
    """
    text = list_note.text()
    links = links_from_text(text)
    if as_dict:
        return {link.file_name: VaultNote(list_note.vault, name=link.file_name) for link in links}
    else:
        return [VaultNote(list_note.vault, name=link.file_name, update_cache=False) for link in links]


def notes_linked_in_notes_linked_in_note(
    list_note: VaultNote, # The `VaultNote` with links to notes containing the links to obtain.
    as_dict: bool = True # If `True`, returns a dict. Returns a list otherwise. 
    ) -> Union[dict[str, VaultNote], list[VaultNote]]: # If dict, the keys are the names of the vault notes and the values are the `VaultNote` objects. If list, then the entries are the `VaultNote` objects.
    """Returns a list or dict of VaultNotes of notes 
    linked by notes which are linked by a specified note.
    """
    linked_in_list_note = notes_linked_in_note(list_note, as_dict=False)
    notes_for_each_note = [notes_linked_in_note(note, as_dict=True) 
                           for note in linked_in_list_note]
    all_notes = {}
    for notes_for_note in notes_for_each_note:
        for name, note in notes_for_note.items():
            all_notes[name] = note
    if as_dict:
        return all_notes
    else:
        return [note for _, note in all_notes.items()]     

In [ ]:
#| export
def add_names_to_html_tags_in_info_note(
        info_note: VaultNote,
        def_and_notat_pipeline: Optional[pipelines.text2text_generation.SummarizationPipeline] = None, # A pipeline wrapping an ML model which predicts the naming of both definition and notations.
        def_pipeline: Optional[pipelines.text2text_generation.SummarizationPipeline] = None,  # A pipeline wrapping an ML model which predicts the naming of definitions. 
        notat_pipeline: Optional[pipelines.text2text_generation.SummarizationPipeline] = None, # A pipeline wrapping an ML model which predicts the naming of notations. 
        # summarizer: pipelines.text2text_generation.SummarizationPipeline, # The pipeline with the ML model
        overwrite: bool = False, # If `True`, overwrite pre-existing, nonempty attributes. If `False`, ignore pre-existing, nonempty attributes and only write on attributes that are empty.
        fix_formatting: bool = True, # If `True`, fix the formatting for notation names.
        correct_syntax: bool = True, # If `True`, attempt to fix syntax errors for notation names.
        ) -> None:
    """
    Predict the names of definitions and notations marked with
    HTML tags within `info_note` and write those names in the
    `"definition"` or `"notation"` attributes in each tag.

    Either `def_and_notat_pipeline` or both `def_pipeline` and `notat_pipeline`
    should be provided.

    An `#_auto/notation_notes_linked` tag is added to
    `origin_notation_note` if such a tag is not already
    present.
    """
    raw_info_note_text = info_note.text()
    raw_info_note_text_minus_html_tags, tags_and_locats = remove_html_tags_in_text(
        raw_info_note_text)
    predicted_names = predict_names(
        info_note, def_and_notat_pipeline, def_pipeline,
        notat_pipeline)

    # If somehow a different number of HTML tags were found
    if len(predicted_names) != len(tags_and_locats):
        # TODO: do warning
        warnings.warn(
            "Somehow, an inconsistent number of HTML tags are "
            f"detected in the note: {info_note.name}.\n"
            "This will raise some indexing issues when marking the definition "
            "and notation names")
    new_tags_and_locations = []
    any_preds_written = False
    for name, (tag, start, end) in zip(predicted_names, tags_and_locats):
        if 'definition' in tag.attrs:
            def_or_notat = 'definition'
        elif 'notation' in tag.attrs:
            def_or_notat = 'notation'
            if correct_syntax and math_mode_string_has_soft_or_hard_syntax_errors(name):
                name = extract_valid_notation_from_source(name, tag.text)
            if fix_formatting:
                name = fix_autogen_formatting(name)
        else:
            # tag could be neither a definition nor a notation tag.
            def_or_notat = ''
        if def_or_notat and (tag.attrs[def_or_notat] == "" or overwrite):
            tag[def_or_notat] = name
            any_preds_written = True
        new_tags_and_locations.append((tag, start, end))
    new_info_note_text = add_HTML_tag_data_to_raw_text(
        raw_info_note_text_minus_html_tags, new_tags_and_locations)
    mf = MarkdownFile.from_string(new_info_note_text)
    if any_preds_written:
        mf.add_tags('_auto/def_and_notat_names_added')
    mf.write(info_note)


# def _correct_syntax(
#         name: str,
#         tag: Tag
#         ) -> str:
#     """
#     This is a helper function of `add_names_to_html_tags_in_info_note`.
#     """
#     replacement_candidates = _list_of_candidates_from_math_mode_strings(tag.text)
#     return correct_latex_syntax_error(name, replacement_candidates)

In [ ]:
# notes_linked_in_note(VaultNote(VAULT, name='ml_data_note_type_categorization'))
test_vault = _test_directory() / 'test_vault_3'
index_note = VaultNote(test_vault, name='_index_1_chapter_reference_1')

notes_as_dict = notes_linked_in_note(index_note)
assert isinstance(notes_as_dict, dict)
test_eq(len(notes_as_dict), 12)
for name, note in notes_as_dict.items():
    assert note.name == name
print(notes_as_dict)

notes_as_list = notes_linked_in_note(index_note)
test_eq(len(notes_as_list), 12)
print(notes_as_list)
# print(notes)

{'note_11': <trouver.obsidian.vault.VaultNote object>, 'note_12': <trouver.obsidian.vault.VaultNote object>, 'note_13': <trouver.obsidian.vault.VaultNote object>, 'note_14': <trouver.obsidian.vault.VaultNote object>, 'note_15': <trouver.obsidian.vault.VaultNote object>, 'note_16': <trouver.obsidian.vault.VaultNote object>, 'note_17': <trouver.obsidian.vault.VaultNote object>, 'note_18': <trouver.obsidian.vault.VaultNote object>, 'note_19': <trouver.obsidian.vault.VaultNote object>, 'note_110': <trouver.obsidian.vault.VaultNote object>, 'note_111': <trouver.obsidian.vault.VaultNote object>, 'note_112': <trouver.obsidian.vault.VaultNote object>}
{'note_11': <trouver.obsidian.vault.VaultNote object>, 'note_12': <trouver.obsidian.vault.VaultNote object>, 'note_13': <trouver.obsidian.vault.VaultNote object>, 'note_14': <trouver.obsidian.vault.VaultNote object>, 'note_15': <trouver.obsidian.vault.VaultNote object>, 'note_16': <trouver.obsidian.vault.VaultNote object>, 'note_17': <trouver.obs

In [ ]:
test_vault = _test_directory() / 'test_vault_3'
index_note = VaultNote(test_vault, name='_index_reference_1')

# This gets all the notes linked in the notes linked in `index_reference_1`.
# In other words, this gets all the notes linked in 
# - `_index_1_chapter_reference_1`
# - `_index_2_chapter_reference_1`, and
# - `_index_3_chapter_reference_1`.
notes_as_dict = notes_linked_in_notes_linked_in_note(index_note)
assert isinstance(notes_as_dict, dict)
test_eq(len(notes_as_dict), 17)
for name, note in notes_as_dict.items():
    assert note.name == name
assert 'note_without_a_section' in notes_as_dict
assert "note_titles don't really need much structure" in notes_as_dict


notes_as_list = notes_linked_in_notes_linked_in_note(index_note)
test_eq(len(notes_as_list), 17)

## Tags in notes

In [ ]:
#| export
def note_has_tag(
        note: VaultNote,
        tag: str # Without the hashtag `'#'`.
        ) -> bool:
    """Returns `True` if the note has the specified tag in its
    yaml frontmatter meta.
    """
    mf = MarkdownFile.from_vault_note(note)
    return mf.has_tag(tag)

In [ ]:
test_vault = _test_directory() / 'test_vault_3'
note = VaultNote(test_vault, name='i_am_an_independent_note_who_does_not_need_a_folder_and_that_is_okay_too')

assert note_has_tag(note, "this_is_a_tag")
assert not note_has_tag(note, "this_is_also_a_tag") # Although `#this_is_also_a_tag` is in the note, it is note in the yaml frontmatter meta.
assert note_has_tag(note, "this_is_a_tag/this_is_a_subtag")
assert not note_has_tag(note, "this_tag_is_not_in_the_note")